In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -o -q "/content/drive/MyDrive/Colab Notebooks/FacebookNetworkGCN/PatternFlow/recognition/FacebookNetworkGCN/facebook_large.zip"

In [4]:
ls facebook_large

citing.txt                musae_facebook_features.json  README.txt
musae_facebook_edges.csv  musae_facebook_target.csv
